In [77]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np    

In [128]:
def build_model():
    model=Sequential()
    model.add(Dense(12, activation='relu', input_shape=(6,)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(4, activation='sigmoid'))
    model.summary()
    return model
model=build_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 12)                84        
_________________________________________________________________
dense_32 (Dense)             (None, 24)                312       
_________________________________________________________________
dense_33 (Dense)             (None, 32)                800       
_________________________________________________________________
dense_34 (Dense)             (None, 24)                792       
_________________________________________________________________
dense_35 (Dense)             (None, 12)                300       
_________________________________________________________________
dense_36 (Dense)             (None, 4)                 52        
Total params: 2,340
Trainable params: 2,340
Non-trainable params: 0
_________________________________________________________________


In [149]:
df=pd.read_csv('data/train.csv')
from sklearn.utils import resample
df_min_1 = df[df.popularity==1]
df_min_2 = df[df.popularity==2]
df_min_3 = df[df.popularity==3]
df_min_4 = df[df.popularity==4]

df_min_2 = resample(df_min_2, replace=True, n_samples=1000, random_state=512)
df_min_3 = resample(df_min_3, replace=True, n_samples=1000, random_state=512)
df_min_4 = resample(df_min_4, replace=True, n_samples=1000, random_state=512)

train = pd.concat([df_min_1,df_min_2,df_min_3,df_min_4]) 

(4740, 7)


In [158]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

X = train.iloc[:, 0:6].values
y = train.iloc[:, 6].values
# Converting to one hot vectors
y = y - 1
y = to_categorical(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = None)

In [159]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

/home/rockstar/miniconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [ ]:
model.load_weights('weights.h5')
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=1, verbose=1)

Epoch 1/30
3792/3792 [==============================] - 12s - loss: 0.1384 - acc: 0.9877    
Epoch 2/30
3792/3792 [==============================] - 12s - loss: 0.1337 - acc: 0.9860    
Epoch 3/30
3792/3792 [==============================] - 11s - loss: 0.1407 - acc: 0.9866    
Epoch 4/30
3792/3792 [==============================] - 10s - loss: 0.1520 - acc: 0.9854    
Epoch 5/30
3792/3792 [==============================] - 10s - loss: 0.1460 - acc: 0.9858    
Epoch 6/30
3792/3792 [==============================] - 10s - loss: 0.1254 - acc: 0.9870    
Epoch 7/30
3792/3792 [==============================] - 10s - loss: 0.1436 - acc: 0.9858    
Epoch 8/30
3792/3792 [==============================] - 10s - loss: 0.1377 - acc: 0.9877    
Epoch 9/30
3792/3792 [==============================] - 12s - loss: 0.1379 - acc: 0.9867    
Epoch 10/30
3792/3792 [==============================] - 11s - loss: 0.1481 - acc: 0.9862    
Epoch 11/30
3792/3792 [==============================] - 11s - loss: 

In [ ]:
y_pred = model.predict(X_test)
print(y_pred[:5])

In [ ]:
# evaluate model
score=model.evaluate(X_test, y_test, verbose=1)
print(score)

In [ ]:
test = pd.read_csv('data/test.csv')
X_test = test.iloc[:, :].values
X_test = scaler.transform(X_test)
y_test = model.predict(X_test)
y_test = np.argmax(y_test, axis=1) + 1
print(y_test)

In [ ]:
# Saving File
result=pd.read_csv('update.csv')
result['popularity']=y_test
result.head()
result.to_csv('prediction.csv', sep=',',index=False, header=False)

In [ ]:
model.save_weights('weights.h5')